In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [3]:
train_data = pd.read_csv('./data/public_train.csv')
test_data = pd.read_csv('./data/public_test.csv')

C:\Programs\miniconda\envs\dap2020_jupyter\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Train or test?
train_data['train_or_test'] = 'train'
test_data['train_or_test'] = 'test'

In [6]:
df = pd.concat([train_data, test_data], sort=True)
df.head()

,AGE,AME_FLAG,APP_SUB_TYPE,BILL_AMOUNT_MONTH_1,BILL_AMOUNT_MONTH_2,BILL_AMOUNT_MONTH_3,BILL_AMOUNT_MONTH_4,BILL_AMOUNT_MONTH_5,BILL_AMOUNT_MONTH_6,BIRTH_CITY,...,RES_CITY,RES_PHONE_AREA_CODE,RES_STATE,RES_TYPE,RES_ZIP,SEX,S_FLAG,TARGET,VISA_FLAG,train_or_test
0,32,0,W,4181,25988,3177,49174,11823,669,City0,...,City2115,105,State0,1.0,595,female,0,1.0,1,train
1,34,0,C,5100,9861,11021,16883,150,4661,City1,...,City1,20,State1,1.0,230,female,0,1.0,0,train
2,61,0,W,97989,101458,103840,105646,86127,67269,City3,...,City3,,State2,NaN,545,female,0,0.0,0,train
3,36,0,W,44970,48101,46993,47940,50903,49794,City9,...,City16,107,State7,1.0,607,female,0,1.0,0,train
4,29,0,W,390,1780,0,0,0,0,City10,...,City10,32,State3,1.0,384,female,0,0.0,0,train


In [7]:
# csak numerikus változók kiemelése
bemeno_valtozok = []
for att in df.columns:
    if df[att].dtype != 'object':
        bemeno_valtozok.append(att)

In [196]:
bemeno_valtozok.remove('ID')
bemeno_valtozok.remove('TARGET')

In [197]:
bemeno_valtozok

['AGE',
 'AME_FLAG',
 'BILL_AMOUNT_MONTH_1',
 'BILL_AMOUNT_MONTH_2',
 'BILL_AMOUNT_MONTH_3',
 'BILL_AMOUNT_MONTH_4',
 'BILL_AMOUNT_MONTH_5',
 'BILL_AMOUNT_MONTH_6',
 'CARS_FLAG',
 'CONF_CITIZEN_CARD_NUM',
 'CONF_TAX_STATUS',
 'C_PRODUCT',
 'DAY_OF_PAYMENT',
 'EDU_LEVEL',
 'EMAIL_FLAG',
 'L_BALANCE',
 'MATE_PROF_CODE',
 'MCARD_FLAG',
 'MONTHS_IN_RES',
 'M_IN_THE_JOB',
 'M_STATUS',
 'NAT',
 'NUMBERO_OF_BANK_ACCOUNTS',
 'NUMBER_OF_DEPENDANTS',
 'NUMBER_OF_SPECIAL_BANK_ACCOUNTS',
 'OCARDS_FLAG',
 'OCC_TYPE',
 'O_INCOMES',
 'PAY_AMOUNT_MONTH_1',
 'PAY_AMOUNT_MONTH_2',
 'PAY_AMOUNT_MONTH_3',
 'PAY_AMOUNT_MONTH_4',
 'PAY_AMOUNT_MONTH_5',
 'PAY_AMOUNT_MONTH_6',
 'PAY_MONTH_0',
 'PAY_MONTH_2',
 'PAY_MONTH_3',
 'PAY_MONTH_4',
 'PAY_MONTH_5',
 'PAY_MONTH_6',
 'PROF_CODE',
 'P_ASSETS_VALUE',
 'P_MONTHLY_INCOME',
 'RES_TYPE',
 'S_FLAG',
 'VISA_FLAG']

In [198]:
df_num = df[['ID'] + bemeno_valtozok + ['train_or_test', 'TARGET']].copy()

In [199]:
df_num.describe()

,ID,AGE,AME_FLAG,BILL_AMOUNT_MONTH_1,BILL_AMOUNT_MONTH_2,BILL_AMOUNT_MONTH_3,BILL_AMOUNT_MONTH_4,BILL_AMOUNT_MONTH_5,BILL_AMOUNT_MONTH_6,CARS_FLAG,...,PAY_MONTH_4,PAY_MONTH_5,PAY_MONTH_6,PROF_CODE,P_ASSETS_VALUE,P_MONTHLY_INCOME,RES_TYPE,S_FLAG,VISA_FLAG,TARGET
count,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.000000,42244.000000,5.000000e+04,49472.000000,48651.000000,50000.000000,50000.000000,25000.000000
mean,25000.500000,43.24852,0.001740,60641.670080,57355.324260,53292.235900,48868.057480,45637.257120,44628.478760,0.336140,...,-0.170260,-0.215300,-0.248160,8.061784,4.451491e+05,180633.123690,1.252225,0.001320,0.111440,0.424320
std,14433.901067,14.98905,0.041677,72452.016459,69798.265537,66190.142456,63214.005733,61091.066392,60077.818698,0.472392,...,1.074325,1.045413,1.069317,3.220104,2.281880e+06,134589.611245,0.867833,0.036308,0.314679,0.494249
min,1.000000,6.00000,0.000000,-165580.000000,-33350.000000,-61506.000000,-65167.000000,-37594.000000,-51443.000000,0.000000,...,-2.000000,-2.000000,-2.000000,0.000000,0.000000e+00,10000.000000,0.000000,0.000000,0.000000,0.000000
25%,12500.750000,31.00000,0.000000,6136.000000,4989.000000,4144.000000,3517.500000,2475.000000,2160.750000,0.000000,...,-1.000000,-1.000000,-1.000000,9.000000,0.000000e+00,94000.000000,1.000000,0.000000,0.000000,0.000000
50%,25000.500000,41.00000,0.000000,43990.000000,40679.000000,34859.000000,28698.000000,26152.000000,25833.000000,0.000000,...,0.000000,0.000000,0.000000,9.000000,0.000000e+00,133200.000000,1.000000,0.000000,0.000000,0.000000
75%,37500.250000,53.00000,0.000000,86422.000000,82789.000000,78366.000000,70899.000000,66895.000000,65239.000000,1.000000,...,0.000000,0.000000,0.000000,9.000000,0.000000e+00,215576.000000,1.000000,0.000000,0.000000,1.000000
max,50000.000000,106.00000,1.000000,964511.000000,983931.000000,693131.000000,891586.000000,927171.000000,961664.000000,1.000000,...,7.000000,7.000000,7.000000,18.000000,1.683611e+07,999976.400000,5.000000,1.000000,1.000000,1.000000


In [200]:
# hiányzó adatok kitöltése
for att in bemeno_valtozok:
    df_num[att]=df_num[att].fillna(np.nanmedian(df_num[att]))

In [201]:
df_num.describe()

,ID,AGE,AME_FLAG,BILL_AMOUNT_MONTH_1,BILL_AMOUNT_MONTH_2,BILL_AMOUNT_MONTH_3,BILL_AMOUNT_MONTH_4,BILL_AMOUNT_MONTH_5,BILL_AMOUNT_MONTH_6,CARS_FLAG,...,PAY_MONTH_4,PAY_MONTH_5,PAY_MONTH_6,PROF_CODE,P_ASSETS_VALUE,P_MONTHLY_INCOME,RES_TYPE,S_FLAG,VISA_FLAG,TARGET
count,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.00000,50000.000000,50000.000000,25000.000000
mean,25000.500000,43.24852,0.001740,60641.670080,57355.324260,53292.235900,48868.057480,45637.257120,44628.478760,0.336140,...,-0.170260,-0.215300,-0.248160,8.207320,4.451491e+05,180132.229904,1.24542,0.001320,0.111440,0.424320
std,14433.901067,14.98905,0.041677,72452.016459,69798.265537,66190.142456,63214.005733,61091.066392,60077.818698,0.472392,...,1.074325,1.045413,1.069317,2.979254,2.281880e+06,133964.847940,0.85702,0.036308,0.314679,0.494249
min,1.000000,6.00000,0.000000,-165580.000000,-33350.000000,-61506.000000,-65167.000000,-37594.000000,-51443.000000,0.000000,...,-2.000000,-2.000000,-2.000000,0.000000,0.000000e+00,10000.000000,0.00000,0.000000,0.000000,0.000000
25%,12500.750000,31.00000,0.000000,6136.000000,4989.000000,4144.000000,3517.500000,2475.000000,2160.750000,0.000000,...,-1.000000,-1.000000,-1.000000,9.000000,0.000000e+00,94000.000000,1.00000,0.000000,0.000000,0.000000
50%,25000.500000,41.00000,0.000000,43990.000000,40679.000000,34859.000000,28698.000000,26152.000000,25833.000000,0.000000,...,0.000000,0.000000,0.000000,9.000000,0.000000e+00,133200.000000,1.00000,0.000000,0.000000,0.000000
75%,37500.250000,53.00000,0.000000,86422.000000,82789.000000,78366.000000,70899.000000,66895.000000,65239.000000,1.000000,...,0.000000,0.000000,0.000000,9.000000,0.000000e+00,213000.000000,1.00000,0.000000,0.000000,1.000000
max,50000.000000,106.00000,1.000000,964511.000000,983931.000000,693131.000000,891586.000000,927171.000000,961664.000000,1.000000,...,7.000000,7.000000,7.000000,18.000000,1.683611e+07,999976.400000,5.00000,1.000000,1.000000,1.000000


In [202]:
df_train = df_num[df_num['train_or_test'] == 'train'].copy()
df_test = df_num[df_num['train_or_test'] == 'test'].copy()

In [203]:
model = Lasso(alpha=1.0, random_state=42)

In [204]:
selector = RFE(model, n_features_to_select=20)

In [205]:
selector.fit(df_train[bemeno_valtozok], df_train['TARGET'])

RFE(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
                    normalize=False, positive=False, precompute=False,
                    random_state=42, selection='cyclic', tol=0.0001,
                    warm_start=False),
    n_features_to_select=20, step=1, verbose=0)

In [206]:
selector.support_

array([False, False,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False, False, False, False,  True,  True, False, False,
       False])

In [207]:
filterezett_valtozok = [valtozo for (valtozo, mask) in zip(bemeno_valtozok, selector.support_) if mask]

In [208]:
reg_model = LogisticRegression(random_state=42)

In [209]:
reg_model.fit(df_train[filterezett_valtozok], df_train['TARGET'])

C:\Programs\miniconda\envs\dap2020_jupyter\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [210]:
df_test['p1'] = reg_model.predict_proba(df_test[filterezett_valtozok])[:, 1]

In [211]:
submission_df = df_test[['ID', 'p1']].copy()
submission_df.columns = ['Id', 'Predicted']
submission_df

,Id,Predicted
0,25001,0.371158
1,25002,0.455085
2,25003,0.398417
3,25004,0.443101
4,25005,0.392663
...,...,...
24995,49996,0.488690
24996,49997,0.494574
24997,49998,0.434977
24998,49999,0.445267


In [212]:
submission_df.to_csv('khf6.csv', index=False)